<a href="https://colab.research.google.com/github/nicholassolomon/Kaggle_Titanic_Competition/blob/main/Titanic_with_PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic Data through PyCaret

* Great article on missing values: https://www.kdnuggets.com/2022/07/scikitlearn-imputer.html


In [ ]:
!pip install pycaret

In [1]:
# Get data
import os
from pathlib import Path

data_path = Path("data/")

if data_path.is_dir():
  print("Non need to download data -- alread there")
else:
  data_path.mkdir(parents=True, exist_ok=True)

# get data from book repo
!git clone https://github.com/nicholassolomon/Kaggle_Titanic_Competition.git
!mv Kaggle_Titanic_Competition/data/*.csv data
!rm -rf Kaggle_Titanic_Competition

Cloning into 'Kaggle_Titanic_Competition'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 57 (delta 17), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (57/57), 1.11 MiB | 2.64 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import pycaret

print(pycaret.__version__)

3.2.0


## Get Data

In [3]:
import pandas as pd

# use processed data from data prep notebook
df_train = pd.read_csv('/content/data/Processed_Training_Data.csv')
#df_train = pd.read_csv('./data/train.csv')
df_train.head()

,Survived,Pclass,Sex,SibSp,Parch,Embarked,relatives,Age_Class,Age_Class_Emb,Age_Adults,...,Fare_Low,Fare_Very High,Fare_Very Low,travelled_alone_no,travelled_alone_yes,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,0.827377,-0.737695,0.432793,-0.473674,-0.568837,0.059160,0.038742,-0.319537,0,...,0,0,0,1,0,0,0,1,0,0
1,1,-1.566107,1.355574,0.432793,-0.473674,1.005181,0.059160,-0.780348,-0.178032,0,...,0,0,0,1,0,0,0,0,1,0
2,1,0.827377,1.355574,-0.474545,-0.473674,-0.568837,-0.560975,0.389780,-0.149731,0,...,0,0,0,0,1,0,1,0,0,0
3,1,-1.566107,1.355574,0.432793,-0.473674,-0.568837,0.059160,-0.868108,-0.758200,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0.827377,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,1.179617,0.232331,0,...,0,0,1,0,1,0,0,1,0,0


## Setup PyCaret

In [4]:
# initial setup PyCaret
from pycaret.classification import *

s = setup(df_train, target = "Survived", session_id = 42, use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Description,Value
0,Session id,42
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 30)"
4,Transformed data shape,"(891, 30)"
5,Transformed train set shape,"(623, 30)"
6,Transformed test set shape,"(268, 30)"
7,Numeric features,29
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

## Compare Models

In [5]:
# compare
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8218,0.8438,0.6951,0.8183,0.7486,0.6125,0.6201,0.1730
knn,K Neighbors Classifier,0.8186,0.8314,0.6951,0.8151,0.7456,0.6065,0.6153,0.0350
lr,Logistic Regression,0.8169,0.8545,0.7159,0.7898,0.7490,0.6058,0.6094,0.0570
lightgbm,Light Gradient Boosting Machine,0.8155,0.8307,0.6995,0.7993,0.7413,0.5997,0.6071,0.1760
ridge,Ridge Classifier,0.8138,0.0000,0.7243,0.7757,0.7480,0.6009,0.6028,0.0410
lda,Linear Discriminant Analysis,0.8122,0.8549,0.7243,0.7724,0.7464,0.5978,0.5997,0.0480
rf,Random Forest Classifier,0.8042,0.8486,0.7161,0.7681,0.7375,0.5821,0.5866,0.3350
et,Extra Trees Classifier,0.8041,0.8220,0.7284,0.7640,0.7417,0.5845,0.5890,0.2970
ada,Ada Boost Classifier,0.7978,0.8302,0.7074,0.7531,0.7273,0.5674,0.5701,0.1390
xgboost,Extreme Gradient Boosting,0.7946,0.8336,0.6951,0.7534,0.7207,0.5591,0.5625,0.2440


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [6]:
# create model and train
model = s.create_model(estimator=best,
                       fold=10,
                       cross_validation=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8254,0.8686,0.7083,0.8095,0.7556,0.6207,0.6240
1,0.8095,0.8531,0.6667,0.8000,0.7273,0.5828,0.5885
2,0.8254,0.8173,0.6667,0.8421,0.7442,0.6144,0.6240
3,0.8387,0.8785,0.8261,0.7600,0.7917,0.6605,0.6620
4,0.8387,0.8114,0.6250,0.9375,0.7500,0.6379,0.6664
5,0.8871,0.8816,0.7917,0.9048,0.8444,0.7565,0.7606
6,0.7903,0.8487,0.6250,0.7895,0.6977,0.5405,0.5491
7,0.8065,0.7917,0.6667,0.8000,0.7273,0.5792,0.5850
8,0.8065,0.8679,0.7500,0.7500,0.7500,0.5921,0.5921


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## Evaluate Model

In [7]:
# evaluate
s.evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

## Tune Model

In [8]:
# tune model
s.tune_model(model, n_iter=100)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8571,0.8611,0.7500,0.8571,0.8000,0.6897,0.6934
1,0.8254,0.8643,0.7083,0.8095,0.7556,0.6207,0.6240
2,0.8730,0.9145,0.7083,0.9444,0.8095,0.7172,0.7339
3,0.8710,0.8986,0.8261,0.8261,0.8261,0.7235,0.7235
4,0.8387,0.8065,0.6250,0.9375,0.7500,0.6379,0.6664
5,0.8871,0.9128,0.7917,0.9048,0.8444,0.7565,0.7606
6,0.7742,0.8613,0.6250,0.7500,0.6818,0.5090,0.5141
7,0.8065,0.8130,0.6250,0.8333,0.7143,0.5724,0.5860
8,0.8065,0.8580,0.7083,0.7727,0.7391,0.5857,0.5871


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='log_loss', max_depth=9,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.2, min_samples_leaf=2,
                           min_samples_split=5, min_weight_fraction_leaf=0.0,
                           n_estimators=150, n_iter_no_change=None,
                           random_state=42, subsample=0.4, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [9]:
# evaluate again after tuniing
s.evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

## Predictions

In [10]:
# predictions
s.predict_model(model)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.7985,0.8675,0.6699,0.7753,0.7188,0.5631,0.5667


,Pclass,Sex,SibSp,Parch,Embarked,relatives,Age_Class,Age_Class_Emb,Age_Adults,Age_Children,...,travelled_alone_no,travelled_alone_yes,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Survived,prediction_label,prediction_score
625,-1.566107,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,-0.107524,-0.390289,0,0,...,0,1,0,0,1,0,0,0,0,0.7909
566,0.827377,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,-0.224537,-0.446891,0,0,...,0,1,0,0,1,0,0,0,0,0.8965
459,0.827377,-0.737695,-0.474545,-0.473674,2.579199,-0.560975,1.618415,3.840696,0,0,...,0,1,0,0,1,0,0,0,0,0.8707
804,0.827377,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,0.477540,-0.107280,0,0,...,0,1,0,0,1,0,0,1,0,0.8773
338,0.827377,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,2.057214,0.656844,0,0,...,0,1,0,0,1,0,0,1,0,0.9697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0.827377,1.355574,-0.474545,2.008933,-0.568837,0.679295,-1.540932,-1.083661,0,1,...,1,0,0,1,0,0,0,1,1,0.9508
607,-1.566107,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,-1.102134,-0.871404,0,0,...,0,1,0,0,1,0,0,1,0,0.6442
624,0.827377,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,-0.049018,-0.361988,0,0,...,0,1,0,0,1,0,0,0,0,0.8854
103,0.827377,-0.737695,-0.474545,-0.473674,-0.568837,-0.560975,1.004098,0.147428,1,0,...,0,1,0,0,1,0,0,0,0,0.9187


## Save Model

In [11]:
import os
from pathlib import Path

model_path = Path("models/")

if model_path.is_dir():
  print("Models directory present")
else:
  model_path.mkdir(parents=True, exist_ok=True)

s.save_model(model, 'gbc_model')

!mv *.pkl models

Transformation Pipeline and Model Successfully Saved


## Predict using Test Data and Model

In [20]:
# predictions

import pandas as pd

df_test = pd.read_csv('./data/Processed_Testing_Data.csv')

test_s = setup(data = df_test)

trained_model = load_model('./models/gbc_model')

df_predictions = test_s.predict_model(trained_model, data = df_test)

,Description,Value
0,Session id,5881
1,Target,Title_Rare
2,Target type,Binary
3,Original data shape,"(418, 29)"
4,Transformed data shape,"(418, 29)"
5,Transformed train set shape,"(292, 29)"
6,Transformed test set shape,"(126, 29)"
7,Numeric features,28
8,Preprocess,True
9,Imputation type,simple


Transformation Pipeline and Model Successfully Loaded


In [21]:
df_predictions.head()

,Pclass,Sex,SibSp,Parch,Embarked,relatives,Age_Class,Age_Class_Emb,Age_Adults,Age_Children,...,Fare_Very Low,travelled_alone_no,travelled_alone_yes,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,prediction_label,prediction_score
0,0.873482,-0.755929,-0.499470,-0.400248,2.243165,-0.553443,0.019419,1.355425,0,0,...,0,0,1,0,0,1,0,0,0,0.9157
1,0.873482,1.322876,0.616992,-0.400248,-0.677841,0.105643,1.218285,0.264573,0,0,...,0,1,0,0,0,0,1,0,1,0.5341
2,-0.315819,-0.755929,-0.499470,-0.400248,2.243165,-0.553443,-0.330250,0.809999,0,0,...,1,0,1,0,0,1,0,0,0,0.9259
3,0.873482,-0.755929,-0.499470,-0.400248,-0.677841,-0.553443,0.993498,0.147696,0,0,...,1,0,1,0,0,1,0,0,0,0.9356
4,0.873482,1.322876,0.616992,0.619896,-0.677841,0.764728,0.993498,0.147696,0,0,...,1,1,0,0,0,0,1,0,0,0.7004


In [28]:
# Create a submisison dataframe and append the relevant columns
submission = pd.DataFrame()
df_test_original = pd.read_csv('./data/test.csv')

submission['PassengerId'] = df_test_original['PassengerId']
submission['Survived'] = df_predictions["prediction_label"]

submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [29]:
# Save our submission
submission.to_csv('./data/Submission_NSSolomon.csv', index=False)